In [8]:
# Script to build Markdown pages that provide term metadata for complex vocabularies
# Steve Baskauf 2020-06-28 CC0
# This script merges static Markdown header and footer documents with term information tables (in Markdown) generated from data in the rs.tdwg.org repo from the TDWG Github site

import re
import requests   # best library to manage HTTP transactions
import csv        # library to read/write/parse CSV files
import json       # library to convert JSON to Python data structures
import pandas as pd

# -----------------
# Configuration section
# -----------------

# !!!! NOTE !!!!
# There is not currently an example of a complex vocabulary that has the column headers
# used in the sample files. In order to test this script, it uses the Audubon Core files,
# which have headers that differ from the samples. So throughout the code, there are
# pairs of lines where the default header names are commented out and the Audubon Core
# headers are not. To build a page using the sample files, you will need to reverse the
# commenting of these pairs.

# This is the base URL for raw files from the branch of the repo that has been pushed to GitHub
githubBaseUri = 'https://raw.githubusercontent.com/tdwg/rs.tdwg.org/master/'

headerFileName = 'termlist-header.md'
footerFileName = 'termlist-footer.md'
outFileName = '../../docs/format.md'

# This is a Python list of the database names of the term lists to be included in the document.
termLists = ['format']
#termLists = ['pathway']

# NOTE! There may be problems unless every term list is of the same vocabulary type since the number of columns will differ
# However, there probably aren't any circumstances where mixed types will be used to generate the same page.
vocab_type = 3 # 1 is simple vocabulary, 2 is simple controlled vocabulary, 3 is c.v. with broader hierarchy

# Terms in large vocabularies like Darwin and Audubon Cores may be organized into categories using tdwgutility_organizedInClass
# If so, those categories can be used to group terms in the generated term list document.
organized_in_categories = True

# If organized in categories, the display_order list must contain the IRIs that are values of tdwgutility_organizedInClass
# If not organized into categories, the value is irrelevant. There just needs to be one item in the list.
display_order = ['', 'm', 'e']
display_label = ['Concept schemes', 'Media types and physical media concept scheme', 'File extensions concept scheme']
display_comments = ['','','']
display_id = ['general_types', 'media_types', 'file_extensions']

#display_order = ['']
#display_label = ['Vocabulary'] # these are the section labels for the categories in the page
#display_comments = [''] # these are the comments about the category to be appended following the section labels
#display_id = ['Vocabulary'] # these are the fragment identifiers for the associated sections for the categories

# ---------------
# Function definitions
# ---------------

# replace URL with link
#
def createLinks(text):
    def repl(match):
        if match.group(1)[-1] == '.':
            return '<a href="' + match.group(1)[:-1] + '">' + match.group(1)[:-1] + '</a>.'
        return '<a href="' + match.group(1) + '">' + match.group(1) + '</a>'

    pattern = '(https?://[^\s,;\)"]*)'
    result = re.sub(pattern, repl, text)
    return result

In [2]:
term_lists_info = []

frame = pd.read_csv(githubBaseUri + 'term-lists/term-lists.csv', na_filter=False)
for termList in termLists:
    term_list_dict = {'list_iri': termList}
    term_list_dict = {'database': termList}
    for index,row in frame.iterrows():
        if row['database'] == termList:
            term_list_dict['pref_ns_prefix'] = row['vann_preferredNamespacePrefix']
            term_list_dict['pref_ns_uri'] = row['vann_preferredNamespaceUri']
            term_list_dict['list_iri'] = row['list']
    term_lists_info.append(term_list_dict)
print(term_lists_info)

[{'database': 'format', 'pref_ns_prefix': 'acformat', 'pref_ns_uri': 'http://rs.tdwg.org/format/values/', 'list_iri': 'http://rs.tdwg.org/format/values/'}]


In [3]:
# Create column list
column_list = ['pref_ns_prefix', 'pref_ns_uri', 'term_localName', 'label', 'definition', 'usage', 'notes', 'examples', 'term_modified', 'term_deprecated', 'type']
if vocab_type == 2:
    column_list += ['controlled_value_string']
elif vocab_type == 3:
    column_list += ['controlled_value_string', 'skos_broader', 'skos_exactMatch']
if organized_in_categories:
    column_list.append('skos_inScheme')
column_list.append('version_iri')

# Create list of lists metadata table
table_list = []
for term_list in term_lists_info:
    # retrieve versions metadata for term list
    versions_url = githubBaseUri + term_list['database'] + '-versions/' + term_list['database'] + '-versions.csv'
    versions_df = pd.read_csv(versions_url, na_filter=False)
    
    # retrieve current term metadata for term list
    data_url = githubBaseUri + term_list['database'] + '/' + term_list['database'] + '.csv'
    frame = pd.read_csv(data_url, na_filter=False)
    for index,row in frame.iterrows():
        row_list = [term_list['pref_ns_prefix'], term_list['pref_ns_uri'], row['term_localName'], row['label'], row['definition'], row['usage'], row['notes'], row['examples'], row['term_modified'], row['term_deprecated'], row['type']]
        if vocab_type == 2:
            row_list += [row['controlled_value_string']]
        elif vocab_type == 3:
            if row['skos_broader'] =='':
                row_list += [row['controlled_value_string'], '']
            else:
                row_list += [row['controlled_value_string'], term_list['pref_ns_prefix'] + ':' + row['skos_broader']]
            if row['skos_exactMatch'] =='':
                row_list += ['']
            else:
                row_list += [term_list['pref_ns_prefix'] + ':' + row['skos_exactMatch']]
        if organized_in_categories:
            row_list.append(row['skos_inScheme'])

        # Borrowed terms really don't have implemented versions. They may be lacking values for version_status.
        # In their case, their version IRI will be omitted.
        found = False
        for vindex, vrow in versions_df.iterrows():
            if vrow['term_localName']==row['term_localName'] and vrow['version_status']=='recommended':
                found = True
                version_iri = vrow['version']
                # NOTE: the current hack for non-TDWG terms without a version is to append # to the end of the term IRI
                if version_iri[len(version_iri)-1] == '#':
                    version_iri = ''
        if not found:
            version_iri = ''
        row_list.append(version_iri)

        table_list.append(row_list)

# Turn list of lists into dataframe
terms_df = pd.DataFrame(table_list, columns = column_list)

#terms_sorted_by_label = terms_df.sort_values(by='label')
# make case insensitive
terms_sorted_by_label = terms_df.iloc[terms_df.label.str.lower().argsort()]
terms_sorted_by_localname = terms_df.sort_values(by='term_localName')
terms_sorted_by_label

,pref_ns_prefix,pref_ns_uri,term_localName,label,definition,usage,notes,examples,term_modified,term_deprecated,type,controlled_value_string,skos_broader,skos_inScheme,version_iri
29,acformat,http://rs.tdwg.org/format/values/,m028,3GPP,media subtype for 3GPP video,,,,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,‎video/3gpp,acformat:m006,m,http://rs.tdwg.org/format/values/version/m028-...
94,acformat,http://rs.tdwg.org/format/values/,e021,3GPP,file extension for 3GPP video,,,,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,3gp,,e,http://rs.tdwg.org/format/values/version/e021-...
32,acformat,http://rs.tdwg.org/format/values/,m031,AIFF,media subtype for Audio Interchange File Forma...,,,,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,audio/x-aiff,acformat:m002,m,http://rs.tdwg.org/format/values/version/m031-...
98,acformat,http://rs.tdwg.org/format/values/,e025,AIFF,file extension for Audio Interchange File Form...,,,,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,aiff,,e,http://rs.tdwg.org/format/values/version/e025-...
31,acformat,http://rs.tdwg.org/format/values/,m030,AMR,media subtype for Adaptive Multi-Rate audio,,,,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,audio/AMR,acformat:m002,m,http://rs.tdwg.org/format/values/version/m030-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,acformat,http://rs.tdwg.org/format/values/,m017,X3D VRML,media subtype for X3D VRML models,,,,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,model/x3d-vrml,acformat:m004,m,http://rs.tdwg.org/format/values/version/m017-...
69,acformat,http://rs.tdwg.org/format/values/,m068,X3D XML,media subtype for X3D XML models,,,,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,model/x3d+xml,acformat:m004,m,http://rs.tdwg.org/format/values/version/m068-...
117,acformat,http://rs.tdwg.org/format/values/,e044,X3D XML,file extension for X3D XML models,,,,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,x3d,,e,http://rs.tdwg.org/format/values/version/e044-...
55,acformat,http://rs.tdwg.org/format/values/,m054,ZVI,unofficial media subtype for Zeiss Vision Imag...,,"This media subtype is in use, but there is no ...",,2020-10-13,,http://www.w3.org/2004/02/skos/core#Concept,image/zvi,acformat:m003,m,http://rs.tdwg.org/format/values/version/m054-...


Run the following cell to generate an index sorted alphabetically by lowercase term local name. Omit this index if the terms have opaque local names.

In [ ]:
# generate the index of terms grouped by category and sorted alphabetically by lowercase term local name

text = '### 3.1 Index By Term Name\n\n'
text += '(See also [3.2 Index By Label](#32-index-by-label))\n\n'
for category in range(0,len(display_order)):
    text += '**' + display_label[category] + '**\n'
    text += '\n'
    if organized_in_categories:
        filtered_table = terms_sorted_by_localname[terms_sorted_by_localname['skos_inScheme']==display_order[category]]
        filtered_table.reset_index(drop=True, inplace=True)
    else:
        filtered_table = terms_sorted_by_localname
        filtered_table.reset_index(drop=True, inplace=True)
        
    for row_index,row in filtered_table.iterrows():
        curie = row['pref_ns_prefix'] + ":" + row['term_localName']
        curie_anchor = curie.replace(':','_')
        text += '[' + row['label'] + '](#' + curie_anchor + ') |\n'
    text = text[:len(text)-2] # remove final trailing vertical bar and newline
    text += '\n\n' # put back removed newline

index_by_name = text

print(index_by_name)

Run the following cell to generate an index by term label

In [4]:
text = '\n\n'

# Comment out the following two lines if there is no index by local names
#text = '### 3.2 Index By Label\n\n'
#text += '(See also [3.1 Index By Term Name](#31-index-by-term-name))\n\n'
for category in range(0,len(display_order)):
    if organized_in_categories:
        text += '**' + display_label[category] + '**\n'
        text += '\n'
        filtered_table = terms_sorted_by_label[terms_sorted_by_label['skos_inScheme']==display_order[category]]
        filtered_table.reset_index(drop=True, inplace=True)
    else:
        filtered_table = terms_sorted_by_label
        filtered_table.reset_index(drop=True, inplace=True)
        
    for row_index,row in filtered_table.iterrows():
        if row_index == 0 or (row_index != 0 and row['label'] != filtered_table.iloc[row_index - 1].loc['label']): # this is a hack to prevent duplicate labels
            curie_anchor = row['pref_ns_prefix'] + "_" + row['term_localName']
            text += '[' + row['label'] + '](#' + curie_anchor + ') |\n'
    text = text[:len(text)-2] # remove final trailing vertical bar and newline
    text += '\n\n' # put back removed newline

index_by_label = text

print(index_by_label)



**Concept schemes**

[file extensions concept scheme](#acformat_e) |
[media types and physical media concept scheme](#acformat_m) 

**Media types and physical media concept scheme**

[3GPP](#acformat_m028) |
[AIFF](#acformat_m031) |
[AMR](#acformat_m030) |
[application](#acformat_m001) |
[ARW raw image](#acformat_m071) |
[ASF](#acformat_m041) |
[audio](#acformat_m002) |
[audio cassette](#acformat_m057) |
[audio CD](#acformat_m044) |
[audio DVD](#acformat_m047) |
[AVI](#acformat_m029) |
[Blu-ray video disc](#acformat_m052) |
[BMP](#acformat_m009) |
[CIFF raw image](#acformat_m069) |
[CR2](#acformat_m023) |
[CR3 raw image](#acformat_m070) |
[data CD](#acformat_m043) |
[data DVD](#acformat_m046) |
[DICOM](#acformat_m066) |
[digital audio tape](#acformat_m058) |
[Digital Negative](#acformat_m022) |
[extended MPEG-2 Transport Stream](#acformat_m037) |
[FLAC](#acformat_m038) |
[GIF](#acformat_m010) |
[gLTF](#acformat_m064) |
[gLTF binary](#acformat_m063) |
[image](#acformat_m003) |
[JPEG](

In [5]:
decisions_df = pd.read_csv('https://raw.githubusercontent.com/tdwg/rs.tdwg.org/master/decisions/decisions-links.csv', na_filter=False)

# generate a table for each term, with terms grouped by category

# generate the Markdown for the terms table
text = '## 4 Vocabulary\n'
for category in range(0,len(display_order)):
    if organized_in_categories:
        text += '### 4.' + str(category + 1) + ' ' + display_label[category] + '\n'
        text += '\n'
        text += display_comments[category] # insert the comments for the category, if any.
        filtered_table = terms_sorted_by_localname[terms_sorted_by_localname['skos_inScheme']==display_order[category]]
        filtered_table.reset_index(drop=True, inplace=True)
    else:
        filtered_table = terms_sorted_by_localname
        filtered_table.reset_index(drop=True, inplace=True)

    for row_index,row in filtered_table.iterrows():
        text += '<table>\n'
        curie = row['pref_ns_prefix'] + ":" + row['term_localName']
        curieAnchor = curie.replace(':','_')
        text += '\t<thead>\n'
        text += '\t\t<tr>\n'
        text += '\t\t\t<th colspan="2"><a id="' + curieAnchor + '"></a>Term Name  ' + curie + '</th>\n'
        text += '\t\t</tr>\n'
        text += '\t</thead>\n'
        text += '\t<tbody>\n'
        text += '\t\t<tr>\n'
        text += '\t\t\t<td>Term IRI</td>\n'
        uri = row['pref_ns_uri'] + row['term_localName']
        text += '\t\t\t<td><a href="' + uri + '">' + uri + '</a></td>\n'
        text += '\t\t</tr>\n'
        text += '\t\t<tr>\n'
        text += '\t\t\t<td>Modified</td>\n'
        text += '\t\t\t<td>' + row['term_modified'] + '</td>\n'
        text += '\t\t</tr>\n'

        if row['version_iri'] != '':
            text += '\t\t<tr>\n'
            text += '\t\t\t<td>Term version IRI</td>\n'
            text += '\t\t\t<td><a href="' + row['version_iri'] + '">' + row['version_iri'] + '</a></td>\n'
            text += '\t\t</tr>\n'

        text += '\t\t<tr>\n'
        text += '\t\t\t<td>Label</td>\n'
        text += '\t\t\t<td>' + row['label'] + '</td>\n'
        text += '\t\t</tr>\n'

        if row['term_deprecated'] != '':
            text += '\t\t<tr>\n'
            text += '\t\t\t<td></td>\n'
            text += '\t\t\t<td><strong>This term is deprecated and should no longer be used.</strong></td>\n'
            text += '\t\t</tr>\n'

        text += '\t\t<tr>\n'
        text += '\t\t\t<td>Definition</td>\n'
        text += '\t\t\t<td>' + row['definition'] + '</td>\n'
        text += '\t\t</tr>\n'

        if row['usage'] != '':
            text += '\t\t<tr>\n'
            text += '\t\t\t<td>Usage</td>\n'
            text += '\t\t\t<td>' + createLinks(row['usage']) + '</td>\n'
            text += '\t\t</tr>\n'

        if row['notes'] != '':
            text += '\t\t<tr>\n'
            text += '\t\t\t<td>Notes</td>\n'
            text += '\t\t\t<td>' + createLinks(row['notes']) + '</td>\n'
            text += '\t\t</tr>\n'

        if row['examples'] != '':
            text += '\t\t<tr>\n'
            text += '\t\t\t<td>Examples</td>\n'
            text += '\t\t\t<td>' + createLinks(row['examples']) + '</td>\n'
            text += '\t\t</tr>\n'

        if (vocab_type == 2 or vocab_type == 3) and row['controlled_value_string'] != '': # controlled vocabulary
            text += '\t\t<tr>\n'
            text += '\t\t\t<td>Controlled value</td>\n'
            text += '\t\t\t<td>' + row['controlled_value_string'] + '</td>\n'
            text += '\t\t</tr>\n'

        if vocab_type == 3 and row['skos_broader'] != '': # controlled vocabulary with skos:broader relationships
            text += '\t\t<tr>\n'
            text += '\t\t\t<td>Has broader concept</td>\n'
            curieAnchor = row['skos_broader'].replace(':','_')
            text += '\t\t\t<td><a href="#' + curieAnchor + '">' + row['skos_broader'] + '</a></td>\n'
            text += '\t\t</tr>\n'

        if vocab_type == 3 and row['skos_exactMatch'] != '': # controlled vocabulary with skos:exactMatch relationships
            text += '\t\t<tr>\n'
            text += '\t\t\t<td>Has exact match</td>\n'
            curieAnchor = row['skos_exactMatch'].replace(':','_')
            text += '\t\t\t<td><a href="#' + curieAnchor + '">' + row['skos_exactMatch'] + '</a></td>\n'
            text += '\t\t</tr>\n'

        text += '\t\t<tr>\n'
        text += '\t\t\t<td>Type</td>\n'
        if row['type'] == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property':
            text += '\t\t\t<td>Property</td>\n'
        elif row['type'] == 'http://www.w3.org/2000/01/rdf-schema#Class':
            text += '\t\t\t<td>Class</td>\n'
        elif row['type'] == 'http://www.w3.org/2004/02/skos/core#Concept':
            text += '\t\t\t<td>Concept</td>\n'
        else:
            text += '\t\t\t<td>' + row['type'] + '</td>\n' # this should rarely happen
        text += '\t\t</tr>\n'

        # Look up decisions related to this term
        for drow_index,drow in decisions_df.iterrows():
            if drow['linked_affected_resource'] == uri:
                text += '\t\t<tr>\n'
                text += '\t\t\t<td>Executive Committee decision</td>\n'
                text += '\t\t\t<td><a href="http://rs.tdwg.org/decisions/' + drow['decision_localName'] + '">http://rs.tdwg.org/decisions/' + drow['decision_localName'] + '</a></td>\n'
                text += '\t\t</tr>\n'                        

        text += '\t</tbody>\n'
        text += '</table>\n'
        text += '\n'
    text += '\n'
term_table = text

print(term_table)

## 4 Vocabulary
### 4.1 Concept schemes

<table>
	<thead>
		<tr>
			<th colspan="2"><a id="acformat_e"></a>Term Name  acformat:e</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td>Term IRI</td>
			<td><a href="http://rs.tdwg.org/format/values/e">http://rs.tdwg.org/format/values/e</a></td>
		</tr>
		<tr>
			<td>Modified</td>
			<td>2020-10-13</td>
		</tr>
		<tr>
			<td>Term version IRI</td>
			<td><a href="http://rs.tdwg.org/format/values/version/e-2020-10-13">http://rs.tdwg.org/format/values/version/e-2020-10-13</a></td>
		</tr>
		<tr>
			<td>Label</td>
			<td>file extensions concept scheme</td>
		</tr>
		<tr>
			<td>Definition</td>
			<td>concept scheme based on conventional file extensions for media types</td>
		</tr>
		<tr>
			<td>Type</td>
			<td>http://www.w3.org/2004/02/skos/core#ConceptScheme</td>
		</tr>
	</tbody>
</table>

<table>
	<thead>
		<tr>
			<th colspan="2"><a id="acformat_m"></a>Term Name  acformat:m</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td>Term IRI</td>
			<td><a hre

Modify to display the indices that you want

In [6]:
text = index_by_label + term_table
#text = index_by_name + index_by_label + term_table

In [7]:
# read in header and footer, merge with terms table, and output

headerObject = open(headerFileName, 'rt', encoding='utf-8')
header = headerObject.read()
headerObject.close()

footerObject = open(footerFileName, 'rt', encoding='utf-8')
footer = footerObject.read()
footerObject.close()

output = header + text + footer
outputObject = open(outFileName, 'wt', encoding='utf-8')
outputObject.write(output)
outputObject.close()
    
print('done')

done
